## ***Flujo del EDA en Python***

# Paso 1. Conexion y carga de la base de datos

In [4]:
# Configuración de la conexión a la base de datos

HOST = "localhost"
PORT = "5432"
DATABSE = "2. PremierLeague"
USER = "admin"
PASSWORD = "GadumUNITEC123"

In [19]:
# Imports

import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import missingno as msno
import psycopg2

In [6]:
# Conexion a la base de datos

def get_engine(host, port, db, user, password):
    url = f"postgresql://{user}:{password}@{host}:{port}/{db}"
    engine = create_engine(url)
    return engine

engine = None
try:
    engine = get_engine(HOST, PORT, DATABSE, USER, PASSWORD)
    print("Engine creado. Prueba de conexión")
    with engine.connect() as connection:
        result = connection.execute("SELECT 1")
        print("Conexión exitosa:", result.fetchone())
except Exception as e:
    print("Error al conectar a la base de datos:", e)

Engine creado. Prueba de conexión
Error al conectar a la base de datos: Not an executable object: 'SELECT 1'


In [9]:
# Consultas a la base de datos

q_team_season = """
SELECT 
id, season_id, team_id, team_name, matches_played, ttl_gls, ttl_xg, ttl_sho AS ttl_sh, ttl_sot, pct_sot, gls_per_sot, ttl_gls_xg_diff, ttl_gls_ag, 
ttl_saves, clean_sheets, avg_poss, ttl_pass_prog, ttl_key_passes, ttl_cross_opp_box, ttl_take_on_suc, ttl_carries, avg_age, ttl_yellow_cards, 
ttl_red_cards, ttl_offside
FROM team_season_stats;
"""

q_team_match = """
SELECT
id, match_id, season_id, team_id, team_name, home_away, ttl_gls, ttl_xg, ttl_sh AS ttl_sh, ttl_sot, pct_sot, ttl_gls_ag, ttl_saves, clean_sheets,
avg_poss, ttl_pass_prog, ttl_key_passes, ttl_cross_opp_box, ttl_take_on_suc, ttl_carries, ttl_yellow_cards, ttl_red_cards
FROM team_match_stats;
"""

q_matches = """
SELECT 
match_id, season_id, matchday, home_team, home_team_id, home_team_score, away_team, away_team_id, away_team_score, date_game
FROM matches_registered;
"""

q_team_meta = "SELECT team_id, team_name FROM team_meta;"
q_data_league = "SELECT season_id, competition_name, squads_count, champion FROM data_league;"

In [10]:
# Carga de datos en DataFrames

try: 
    df_season = pd.read_sql(q_team_season, engine)
    df_match = pd.read_sql(q_team_match, engine)
    df_matches_reg = pd.read_sql(q_matches, engine)
    df_meta = pd.read_sql(q_team_meta, engine)
    df_league = pd.read_sql(q_data_league, engine)
    print("Tablas cargas: ", list(map(len, [df_season, df_match, df_matches_reg, df_meta, df_league])))
except Exception as e:
    print("No se pudieron cargar las tablas automaticamente. Error:", e)
    df_season = pd.DataFrame()
    df_match = pd.DataFrame()
    df_matches_reg = pd.DataFrame()
    df_meta = pd.DataFrame()
    df_league = pd.DataFrame()

Tablas cargas:  [160, 6070, 3038, 32, 127]


In [11]:
# Utilidades rapidas

def show_head(df, name='df'):
    print(f"--- {name} HEAD (n={len(df)}) ---")
    display(df.head())

def save_fig(fig, fname):
    outdir = "eda_outputs"
    os.makedirs(outdir, exist_ok=True)
    path = os.path.join(outdir, fname)
    fig.savefig(path, bbox_inches='tight', dpi = 150)
    print("Guardado en: ", path)

In [13]:
# Revisión inicial y resumen de columnas
for name, df in [('team_season_stats', df_season), ('team_match_stats', df_match), ('matches_registered', df_matches_reg)]:
    print('\n', name, '-> filas:', len(df), 'columnas:', list(df.columns)[:10])


print('\nNulos por tabla:')
print('team_season_stats:\n', df_season.isnull().sum())
print('\nteam_match_stats:\n', df_match.isnull().sum())
print('\nmatches_registered:\n', df_matches_reg.isnull().sum())


 team_season_stats -> filas: 160 columnas: ['id', 'season_id', 'team_id', 'team_name', 'matches_played', 'ttl_gls', 'ttl_xg', 'ttl_sh', 'ttl_sot', 'pct_sot']

 team_match_stats -> filas: 6070 columnas: ['id', 'match_id', 'season_id', 'team_id', 'team_name', 'home_away', 'ttl_gls', 'ttl_xg', 'ttl_sh', 'ttl_sot']

 matches_registered -> filas: 3038 columnas: ['match_id', 'season_id', 'matchday', 'home_team', 'home_team_id', 'home_team_score', 'away_team', 'away_team_id', 'away_team_score', 'date_game']

Nulos por tabla:
team_season_stats:
 id                    0
season_id             0
team_id               0
team_name             0
matches_played        0
ttl_gls               0
ttl_xg                0
ttl_sh                0
ttl_sot               0
pct_sot               0
gls_per_sot           0
ttl_gls_xg_diff       0
ttl_gls_ag            0
ttl_saves             0
clean_sheets          0
avg_poss              0
ttl_pass_prog         0
ttl_key_passes        0
ttl_cross_opp_box     0

# Paso 2. Limpieza inicial

In [29]:
# Cargar tablas principales

team_stats = pd.read_sql('SELECT * FROM team_season_stats;', engine)
matches = pd.read_sql('SELECT * FROM matches_registered;', engine)
teams = pd.read_sql('SELECT * FROM team_meta;', engine)
league = pd.read_sql('SELECT * FROM data_league;', engine)

In [30]:
# Limpieza Inicial

print("Shape inicial team_stats:", team_stats.shape)
print("Shape inicial matches:", matches.shape)

Shape inicial team_stats: (160, 78)
Shape inicial matches: (3038, 11)


In [31]:
# 1. Eliminar duplicados

total_dups_team_stats = team_stats.duplicated(subset=['season_id', 'team_id']).sum()
team_stats = team_stats.drop_duplicates(subset=['season_id', 'team_id'])

In [32]:
# 2. Revisar valores nulos

nulls_team_stats = team_stats.isnull().sum().sort_values(ascending=False)
nulls_matches = matches.isnull().sum().sort_values(ascending=False)

In [33]:
# 3. Validar tipos de datos
print(team_stats.dtypes.head(10))

id                   int64
season_id           object
team_id             object
team_name           object
matches_played       int64
ttl_gls              int64
ttl_ast              int64
ttl_non_pen_gls      int64
ttl_xg             float64
ttl_xag            float64
dtype: object


In [34]:
# 4. Verificar consistencia de IDs
missing_teams = team_stats[~team_stats['team_id'].isin(teams['team_id'])]
print(f"Equipos sin metadatos: {missing_teams.shape[0]}")

Equipos sin metadatos: 0


In [35]:
# 5. Rellenar o eliminar datos faltantes numéricos
numeric_cols = team_stats.select_dtypes(include=['float64','int64']).columns
team_stats[numeric_cols] = team_stats[numeric_cols].fillna(0)

In [36]:
# 6. Guardar dataframe limpio
table_cleaned_path = 'eda_outputs/team_season_stats_cleaned.csv'
team_stats.to_csv(table_cleaned_path, index=False)
print(f"\nDatos limpios guardados en: {table_cleaned_path}")

OSError: Cannot save file into a non-existent directory: 'eda_outputs'

# Paso 3. Estadisticas descriptivas

# Paso 4. Correlaciones

# Paso 5. Comparativas

# Paso 6. Outliers y consistencias 